# Marginal distributions

This notebook is the continuation of `bandwidths.ipynb` notebook, apliying the bw results and the class for get the n-dimentional pdf. 

#### Some considerations: 
1. As **prior** information we will take the cleaned (without NaN or inf values) information from exoplanet.eu, this is the first part of notebook.

2. The pdf is the **likelihood** for a certain number of variables in synthetic systems with no-pertutbation, low perturbation and high perturbation. 

3. To get the **marginal** distributions of a variable of interest, we go in the same way that the example marginalization at the end of notebook `bandwidths.ipynb`.

In [13]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

import nbimporter
from bandwidths import multidim_bw #import the class for get the pdf. 

In [14]:
#========================== Simulation Data ==========================
dn=pd.read_csv('data/proc_no_pert.csv',index_col=None); dn["gia"]=dn.ngi>0 #without pertubtations 
dl=pd.read_csv('data/proc_lo_pert.csv',index_col=None); dl["gia"]=dl.ngi>0 #with low pertubtations 
dh=pd.read_csv('data/proc_hi_pert.csv',index_col=None); dh["gia"]=dh.ngi>0 #with high pertubtations

In [15]:
#============================== variables ===========================
##Terrestrial: t; giant;g
dnt=dn[~dn["gia"]]; dng=dn[dn["gia"]] # without pertubtations 
dlt=dl[~dl["gia"]]; dlg=dl[dl["gia"]] # low pertubtations 
dht=dh[~dh["gia"]]; dhg=dh[dh["gia"]] # high pertubtations 

x_variables = [dng,dlg,dhg,dnt,dlt,dht,dn,dl,dh]

for i, var in enumerate(x_variables):
    var['logeff'] = np.log10(var.massefficiency)
    var['logcom'] = np.log10(var.com)

In [16]:
a = multidim_bw(dng.logeff, dng.logcom)
likelihood = a.pdf_ndim()

array([[1.24319708e-09, 2.99736086e-08, 5.01941172e-07, 5.83823202e-06,
        4.71655194e-05, 2.93083867e-04, 1.11483942e-03, 2.94542161e-03,
        5.40502143e-03, 6.88910167e-03, 6.09884366e-03, 3.58443726e-03,
        1.49911857e-03, 4.63059795e-04, 2.22900824e-04, 4.70684542e-04,
        1.26548378e-03, 2.66756326e-03, 5.00287583e-03, 8.79413459e-03,
        1.43537648e-02, 2.10678977e-02, 2.50413426e-02, 2.38405672e-02,
        1.77683776e-02, 1.02926327e-02, 4.65229604e-03, 1.52874684e-03,
        4.19388635e-04, 8.91174352e-05],
       [1.67516855e-09, 4.03884832e-08, 6.76349742e-07, 7.86683167e-06,
        6.35540350e-05, 3.94921179e-04, 1.50221068e-03, 3.96886271e-03,
        7.28310308e-03, 9.28291743e-03, 8.21849254e-03, 4.83308762e-03,
        2.03560494e-03, 6.91936476e-04, 5.57871436e-04, 1.44485795e-03,
        3.89389553e-03, 8.12496791e-03, 1.46745251e-02, 2.38696486e-02,
        3.52115498e-02, 4.68218436e-02, 5.24057679e-02, 4.88082682e-02,
        3.68202940e-02,